# Cobb-Douglas Production Function: MFP Decomposition

This notebook decomposes Multi-Factor Productivity (MFP) from GDP using a Cobb-Douglas production function framework.

## Important: What This Model Is (and Isn't)

This is a **descriptive accounting framework**, not a structural model of productivity. The MFP residual - sometimes called "the measure of our ignorance" (Abramovitz) - captures everything not explained by measured capital and labour inputs: true technological progress, but also measurement error, capacity utilization changes, quality improvements, terms of trade effects, and unmeasured intangibles. The decomposition is useful for broad narratives about growth sources, but should not be mistaken for a precise measure of "technology" or "efficiency". Results are sensitive to the assumed capital share (α), and the HP filter used for trend extraction has known end-point problems.

## Methodology

The Cobb-Douglas production function:
$$Y = A \cdot K^\alpha \cdot L^{1-\alpha}$$

In growth form (logs):
$$g_Y = g_{MFP} + \alpha \cdot g_K + (1-\alpha) \cdot g_L$$

Solving for MFP (Solow residual):
$$g_{MFP} = g_Y - \alpha \cdot g_K - (1-\alpha) \cdot g_L$$

## Approach

Calculate raw MFP as Solow residual, then apply HP filter to extract trend.

## Important Caveat: Output Gap

The output gap estimates from this model are **notional only**. Unlike joint NAIRU+Output Gap models that use inflation dynamics (Phillips curve) to discipline the gap estimate, this production function approach derives potential GDP purely from filtered factor inputs and productivity. As a result:
- The gap is not cross-checked against inflation outcomes
- Persistent gaps would imply sustained inflation/disinflation that may not match reality
- Periodic re-anchoring mitigates cumulative drift but does not address this fundamental limitation

For policy-relevant output gap estimates, prefer models that incorporate inflation information.

## Data Sources
- **GDP**: ABS 5206.0 - Chain Volume Measures, Seasonally Adjusted (from 1959Q3)
- **Capital Stock**: ABS 1364.0.15.003 - Net capital stock (CVM) (from ~1985)
- **Hours Worked**: ABS 6202.0 - Monthly hours worked, converted to quarterly (from 1978Q3)

## Python Setup

In [1]:
# System imports
import warnings

warnings.filterwarnings("ignore")

# Analytic imports
import matplotlib.pyplot as plt

# Local imports
import mgplot as mg
import numpy as np
import pandas as pd
import readabs as ra
from abs_helper import get_gdp
from abs_structured_capture import ReqsTuple
from abs_structured_capture import get_abs_data as get_abs_data_structured
from readabs import metacol
from statsmodels.tsa.filters.hp_filter import hpfilter

# Display settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
SHOW = False

# Chart directory
CHART_DIR = "./CHARTS/MFP Decomposition/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()

## Model Parameters

In [2]:
# Cobb-Douglas parameters
ALPHA = 0.30  # Capital share of income (typical range: 0.25-0.35)

# Filter parameters
HP_LAMBDA = 1600  # HP filter smoothing parameter for quarterly data

# Sample period
# Capital stock from Modellers Database starts ~1985, hours worked from 6202 starts 1978
START_DATE = "1985Q1"  # Constrained by capital stock availability
END_DATE = None  # Use latest available

print(f"Capital share (alpha): {ALPHA}")
print(f"Labour share (1-alpha): {1-ALPHA}")
print(f"HP filter lambda: {HP_LAMBDA}")

Capital share (alpha): 0.3
Labour share (1-alpha): 0.7
HP filter lambda: 1600


## Fetch Data

### GDP - Chain Volume Measures

In [3]:
def get_gdp_series() -> pd.Series:
    """Fetch GDP Chain Volume Measures, Seasonally Adjusted."""
    gdp, units = get_gdp(gdp_type="CVM", seasonal="SA")
    gdp.name = "GDP (CVM)"
    print(f"GDP data: {gdp.index[0]} to {gdp.index[-1]}")
    print(f"Units: {units}")
    return gdp

gdp = get_gdp_series()
gdp.tail()

GDP data: 1959Q3 to 2025Q3
Units: $ Millions


Series ID
2024Q3    673927.0
2024Q4    677564.0
2025Q1    680085.0
2025Q2    685076.0
2025Q3    687757.0
Freq: Q-DEC, Name: GDP (CVM), dtype: float64

### Capital Stock - Net Capital Stock (CVM)

In [4]:
def get_capital_stock() -> pd.Series:
    """Fetch net capital stock from ABS Modellers Database.
    
    Uses net capital stock of non-financial and financial corporations
    (chain volume measures) from ABS 1364.0.15.003.
    """
    cat = "1364.0.15.003"
    seo = "1364015003"
    did = "Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;"

    data, meta = ra.read_abs_cat(cat, single_excel_only=seo, verbose=False)

    search = {
        did: metacol.did,
        "Seasonally Adjusted": metacol.stype,
    }
    table, series_id, units = ra.find_abs_id(meta, search, verbose=False)
    capital = data[table][series_id]
    capital.name = "Capital Stock (CVM)"

    print(f"Capital stock data: {capital.index[0]} to {capital.index[-1]}")
    print(f"Units: {units}")
    return capital

capital = get_capital_stock()
capital.tail()

Capital stock data: 1959Q3 to 2025Q3
Units: $ Millions


Series ID
2024Q3    3836846.0
2024Q4    3858660.0
2025Q1    3879676.0
2025Q2    3900824.0
2025Q3    3922329.0
Freq: Q-DEC, Name: Capital Stock (CVM), dtype: float64

### Hours Worked - Monthly Labour Force Survey (converted to quarterly)

In [5]:
def get_hours_worked() -> pd.Series:
    """Fetch total hours worked from ABS Monthly Labour Force Survey.
    
    Uses monthly hours worked in all jobs from ABS 6202.0, converted to quarterly.
    This provides a longer history (from 1978) than the Labour Account (from 1994).
    """
    cat = "6202.0"

    data, meta = ra.read_abs_cat(cat, verbose=False)

    # Monthly hours worked in all jobs - Persons - Seasonally Adjusted
    table = "6202019"
    did = "Monthly hours worked in all jobs ;  Persons ;"
    series_type = "Seasonally Adjusted"

    search = {
        table: metacol.table,
        did: metacol.did,
        series_type: metacol.stype,
    }
    _table, series_id, units = ra.find_abs_id(meta, search, verbose=False)
    hours_monthly = data[table][series_id]

    # Convert monthly to quarterly (sum of 3 months)
    hours_quarterly = ra.monthly_to_qtly(hours_monthly, f="sum")
    hours_quarterly.name = "Hours Worked"

    print(f"Hours worked data: {hours_quarterly.index[0]} to {hours_quarterly.index[-1]}")
    print(f"Units: {units} (quarterly sum)")
    return hours_quarterly

hours = get_hours_worked()
hours.tail()

Hours worked data: 1978Q3 to 2025Q3
Units: 000 Hours (quarterly sum)


Series ID
2024Q3    5.868244e+06
2024Q4    5.909717e+06
2025Q1    5.930158e+06
2025Q2    5.952394e+06
2025Q3    5.957746e+06
Freq: Q-DEC, Name: Hours Worked, dtype: float64

### Align Data to Common Sample

In [6]:
def align_data(gdp: pd.Series, capital: pd.Series, hours: pd.Series) -> pd.DataFrame:
    """Align all series to a common sample period."""
    # Combine into DataFrame
    df = pd.DataFrame({
        "GDP": gdp,
        "Capital": capital,
        "Hours": hours,
    })

    # Apply sample period
    if START_DATE:
        df = df[df.index >= pd.Period(START_DATE)]
    if END_DATE:
        df = df[df.index <= pd.Period(END_DATE)]

    # Drop rows with missing values
    df = df.dropna()

    print(f"\nAligned sample: {df.index[0]} to {df.index[-1]}")
    print(f"Observations: {len(df)}")

    return df

data = align_data(gdp, capital, hours)
data.head()


Aligned sample: 1985Q1 to 2025Q3
Observations: 163


,GDP,Capital,Hours
Series ID,,,
1985Q1,209049.0,945650.0,2.935410e+06
1985Q2,213735.0,955598.0,2.956973e+06
1985Q3,216480.0,966172.0,2.994855e+06
1985Q4,215860.0,977112.0,3.037046e+06
1986Q1,217343.0,987978.0,3.087028e+06


### Plot Raw Data

In [7]:
def plot_raw_data(data: pd.DataFrame) -> None:
    """Plot the raw input data series."""
    fig, axes = plt.subplots(3, 1, figsize=(9.0,9.0))

    # GDP
    gdp_data = pd.DataFrame({"GDP": data["GDP"].dropna()})
    mg.line_plot(gdp_data, ax=axes[0], color="blue", width=1.5)
    mg.finalise_plot(
        axes[0],
        title="GDP (Chain Volume Measures)",
        ylabel="$ Millions",
        dont_save=True,
        dont_close=True,
    )

    # Capital
    capital_data = pd.DataFrame({"Capital Stock": data["Capital"]})
    mg.line_plot(capital_data, ax=axes[1], color="green", width=1.5)
    mg.finalise_plot(
        axes[1],
        title="Net Capital Stock (CVM)",
        ylabel="$ Millions",
        dont_save=True,
        dont_close=True,
    )

    # Hours
    hours_data = pd.DataFrame({"Hours Worked": data["Hours"]})
    mg.line_plot(hours_data, ax=axes[2], color="orange", width=1.5)
    mg.finalise_plot(
        axes[2],
        title="Hours Actually Worked",
        ylabel="Millions of Hours",
        figsize=(9.0, 9.0),
        rfooter="ABS 5206, 1364, 6202",
        lfooter="Australia. ",
        show=SHOW,
    )

plot_raw_data(data)

## Calculate Growth Rates

In [8]:
def calculate_growth_rates(data: pd.DataFrame) -> pd.DataFrame:
    """Calculate quarterly log growth rates (x100 for percentage).
    
    Using log differences ensures growth rates are additive in the
    Cobb-Douglas decomposition.
    """
    growth = pd.DataFrame(index=data.index)

    # Log levels
    growth["log_GDP"] = np.log(data["GDP"]) * 100
    growth["log_Capital"] = np.log(data["Capital"]) * 100
    growth["log_Hours"] = np.log(data["Hours"]) * 100

    # Quarterly growth rates (log difference x 100 = approx percentage)
    growth["g_GDP"] = growth["log_GDP"].diff(1)
    growth["g_Capital"] = growth["log_Capital"].diff(1)
    growth["g_Hours"] = growth["log_Hours"].diff(1)

    # Annual growth rates (4-quarter difference)
    growth["g_GDP_annual"] = growth["log_GDP"].diff(4)
    growth["g_Capital_annual"] = growth["log_Capital"].diff(4)
    growth["g_Hours_annual"] = growth["log_Hours"].diff(4)

    print("Growth rate statistics (quarterly):")
    print(growth[["g_GDP", "g_Capital", "g_Hours"]].describe().round(3))

    return growth

growth = calculate_growth_rates(data)
growth.dropna().tail()

Growth rate statistics (quarterly):
         g_GDP  g_Capital  g_Hours
count  162.000    162.000  162.000
mean     0.735      0.878    0.437
std      0.951      0.380    1.136
min     -6.989      0.306   -9.358
25%      0.376      0.519    0.067
50%      0.726      0.893    0.472
75%      1.040      1.121    0.898
max      3.535      1.726    4.146


,log_GDP,log_Capital,log_Hours,g_GDP,g_Capital,g_Hours,g_GDP_annual,g_Capital_annual,g_Hours_annual
Series ID,,,,,,,,,
2024Q3,1342.087708,1516.016123,1558.506606,0.292298,0.584947,0.708293,0.815571,2.391028,1.276277
2024Q4,1342.625929,1516.583053,1559.210849,0.538222,0.566930,0.704243,1.280665,2.356306,2.600139
2025Q1,1342.997307,1517.126220,1559.556148,0.371378,0.543167,0.345299,1.394058,2.293797,2.243893
2025Q2,1343.728506,1517.669837,1559.930399,0.731199,0.543617,0.374252,1.933097,2.238661,2.132086
2025Q3,1344.119086,1518.219617,1560.020278,0.390580,0.549780,0.089878,2.031378,2.203494,1.513671


## Solow Residual: Raw MFP Calculation

In [9]:
def calculate_solow_residual(
    growth: pd.DataFrame,
    alpha: float = ALPHA
) -> pd.DataFrame:
    """Calculate MFP as the Solow residual.
    
    g_MFP = g_Y - alpha * g_K - (1-alpha) * g_L
    
    This is "raw" MFP - it includes cyclical fluctuations and measurement error.
    """
    result = growth.copy()

    # Capital contribution
    result["contrib_Capital"] = alpha * result["g_Capital"]

    # Labour contribution
    result["contrib_Hours"] = (1 - alpha) * result["g_Hours"]

    # MFP as residual (quarterly)
    result["g_MFP_raw"] = (
        result["g_GDP"]
        - result["contrib_Capital"]
        - result["contrib_Hours"]
    )

    # Annual equivalents
    result["contrib_Capital_annual"] = alpha * result["g_Capital_annual"]
    result["contrib_Hours_annual"] = (1 - alpha) * result["g_Hours_annual"]
    result["g_MFP_raw_annual"] = (
        result["g_GDP_annual"]
        - result["contrib_Capital_annual"]
        - result["contrib_Hours_annual"]
    )

    # Verify decomposition (should sum to GDP growth)
    check = (
        result["contrib_Capital"]
        + result["contrib_Hours"]
        + result["g_MFP_raw"]
        - result["g_GDP"]
    ).dropna().abs().max()
    print(f"Decomposition check (max error): {check:.2e}")

    print(f"\nRaw MFP statistics (quarterly, alpha={alpha}):")
    print(f"  Mean: {result['g_MFP_raw'].dropna().mean():.3f}%")
    print(f"  Std:  {result['g_MFP_raw'].dropna().std():.3f}%")
    print(f"  Annual mean: {result['g_MFP_raw'].dropna().mean() * 4:.2f}%")

    return result

decomp = calculate_solow_residual(growth)
decomp[["g_GDP", "contrib_Capital", "contrib_Hours", "g_MFP_raw"]].dropna().tail(10)

Decomposition check (max error): 2.22e-16

Raw MFP statistics (quarterly, alpha=0.3):
  Mean: 0.166%
  Std:  0.699%
  Annual mean: 0.66%


,g_GDP,contrib_Capital,contrib_Hours,g_MFP_raw
Series ID,,,,
2023Q2,0.395257,0.157346,0.981897,-0.743987
2023Q3,0.375900,0.171777,-0.732405,0.936528
2023Q4,0.073127,0.180496,-0.433733,0.326365
2024Q1,0.257985,0.181703,0.491081,-0.414799
2024Q2,0.192160,0.179626,0.340241,-0.327706
2024Q3,0.292298,0.175484,0.495805,-0.378991
2024Q4,0.538222,0.170079,0.492970,-0.124827
2025Q1,0.371378,0.162950,0.241709,-0.033282
2025Q2,0.731199,0.163085,0.261976,0.306138


## Filter Approach: Extract Trend MFP

### HP Filter

HP filter (Hodrick-Prescott filter) is a popular tool in macroeconomics that separates a time series (like GDP) into its long-term trend component (growth) and its short-term cyclical component (fluctuations), by minimizing a penalty function that balances how well the trend fits the data against how smooth the trend is

In [10]:
def apply_hp_filter(
    series: pd.Series,
    lamb: float = HP_LAMBDA
) -> tuple[pd.Series, pd.Series]:
    """Apply Hodrick-Prescott filter to extract trend and cycle.
    
    HP filter minimizes:
    sum((y_t - tau_t)^2) + lambda * sum((tau_{t+1} - 2*tau_t + tau_{t-1})^2)
    
    lambda=1600 is standard for quarterly data (Ravn-Uhlig rule).
    """
    # Drop NaN for filter
    clean = series.dropna()

    # Apply HP filter
    cycle, trend = hpfilter(clean, lamb=lamb)

    # Convert back to Series with original index
    trend = pd.Series(trend, index=clean.index, name=f"{series.name}_HP_trend")
    cycle = pd.Series(cycle, index=clean.index, name=f"{series.name}_HP_cycle")

    return trend, cycle

# Apply to raw MFP
mfp_hp_trend, mfp_hp_cycle = apply_hp_filter(decomp["g_MFP_raw"])
print(f"HP Filter (lambda={HP_LAMBDA}):")
print(f"  Trend MFP mean: {mfp_hp_trend.mean():.3f}% quarterly ({mfp_hp_trend.mean()*4:.2f}% annual)")
print(f"  Cycle std: {mfp_hp_cycle.std():.3f}%")

HP Filter (lambda=1600):
  Trend MFP mean: 0.166% quarterly (0.66% annual)
  Cycle std: 0.670%


## Cross-Check: Compare All Methods

In [11]:
def cross_check_hp_filter(
    raw: pd.Series,
    hp_trend: pd.Series,
) -> pd.DataFrame:
    """Summarize HP filter trend extraction."""
    comparison = pd.DataFrame({
        "Raw_MFP": raw,
        "HP_Filter": hp_trend,
    })

    print("=" * 60)
    print("HP Filter Trend Extraction Summary")
    print("=" * 60)

    print("\n1. Mean quarterly growth (% per quarter):")
    for col in comparison.columns:
        mean = comparison[col].dropna().mean()
        print(f"   {col:15s}: {mean:6.3f}% quarterly ({mean*4:5.2f}% annual)")

    print("\n2. Standard deviation (quarterly):")
    for col in comparison.columns:
        std = comparison[col].dropna().std()
        print(f"   {col:15s}: {std:6.3f}%")

    # Recent period comparison
    recent = comparison.loc["2015Q1":].dropna()
    print("\n3. Recent period (2015-present) mean annual growth:")
    for col in comparison.columns:
        mean = recent[col].dropna().mean() * 4
        print(f"   {col:15s}: {mean:5.2f}% p.a.")

    return comparison

comparison = cross_check_hp_filter(
    decomp["g_MFP_raw"].dropna(),
    mfp_hp_trend,
)

HP Filter Trend Extraction Summary

1. Mean quarterly growth (% per quarter):
   Raw_MFP        :  0.166% quarterly ( 0.66% annual)
   HP_Filter      :  0.166% quarterly ( 0.66% annual)

2. Standard deviation (quarterly):
   Raw_MFP        :  0.699%
   HP_Filter      :  0.179%

3. Recent period (2015-present) mean annual growth:
   Raw_MFP        :  0.29% p.a.
   HP_Filter      :  0.31% p.a.


In [12]:
def plot_mfp_trend(comparison: pd.DataFrame) -> None:
    """Plot raw MFP vs HP filter trend."""
    fig, axes = plt.subplots(2, 1, figsize=(9.0, 9.0))

    # Panel 1: Full sample - quarterly
    panel1_data = pd.DataFrame({
        "Raw MFP": comparison["Raw_MFP"],
        f"HP Filter (λ={HP_LAMBDA})": comparison["HP_Filter"],
    })

    mg.line_plot(
        panel1_data,
        ax=axes[0],
        color=["gray", "blue"],
        alpha=[0.3, 1.0],
        width=[0.8, 2],
    )

    mg.finalise_plot(
        axes[0],
        title=f"Multi-Factor Productivity Growth (α={ALPHA})",
        ylabel="Quarterly Growth (%)",
        y0=True,
        legend={"loc": "best", "fontsize": "x-small"},
        dont_save=True,
        dont_close=True,
    )

    # Panel 2: Annualized trend
    panel2_data = pd.DataFrame({
        f"HP Filter (λ={HP_LAMBDA})": comparison["HP_Filter"] * 4,
    })

    mg.line_plot(
        panel2_data,
        ax=axes[1],
        color="blue",
        width=2,
    )

    # Add reference line at 1% growth
    axes[1].axhline(y=1.0, color="gray", linewidth=0.5, linestyle="--", alpha=0.5)

    mg.finalise_plot(
        axes[1],
        title="Trend MFP Growth (Annualised)",
        ylabel="Annual Growth (% p.a.)",
        y0=True,
        legend={"loc": "best", "fontsize": "x-small"},
        figsize=(9.0, 9.0),
        lfooter="Australia. MFP (Solow residual) = GDP growth − α×Capital growth − (1−α)×Hours growth",
        rfooter="ABS 5206, 1364, 6202",
        show=SHOW,
    )

plot_mfp_trend(comparison)

## Potential GDP Projection

In [13]:
# Anchor points for potential GDP re-anchoring
# Use business cycle peaks or decade boundaries to prevent cumulative drift
ANCHOR_POINTS = ["1990Q1", "2000Q1", "2008Q1", "2019Q4"]  # Pre-recession peaks

def calculate_potential_gdp(
    data: pd.DataFrame,
    growth: pd.DataFrame,
    mfp_trend: pd.Series,
    alpha: float = ALPHA,
    method_name: str = "HP",
    anchor_points: list = None,
) -> pd.DataFrame:
    """Calculate potential GDP using trend MFP with periodic re-anchoring.
    
    Potential GDP growth = alpha * g_K_trend + (1-alpha) * g_L_trend + g_MFP_trend
    
    Re-anchoring at specified points prevents cumulative drift from compounding
    small biases in trend growth into large level gaps over long samples.
    
    Parameters
    ----------
    anchor_points : list of str
        Period strings (e.g., "2000Q1") where potential GDP is re-anchored to actual.
        If None, only anchors at start of sample (original behavior).

    """
    if anchor_points is None:
        anchor_points = []

    result = pd.DataFrame(index=growth.index)

    # Use HP-filtered capital and hours growth for potential
    g_K_trend, _ = apply_hp_filter(growth["g_Capital"].dropna())
    g_L_trend, _ = apply_hp_filter(growth["g_Hours"].dropna())

    # Potential GDP growth (quarterly)
    result["g_Y_potential"] = (
        alpha * g_K_trend.reindex(result.index)
        + (1 - alpha) * g_L_trend.reindex(result.index)
        + mfp_trend.reindex(result.index)
    )

    # Actual GDP growth
    result["g_Y_actual"] = growth["g_GDP"]

    # Log GDP levels
    result["log_GDP_actual"] = growth["log_GDP"]

    # Convert anchor points to periods
    anchor_periods = [pd.Period(ap) for ap in anchor_points if pd.Period(ap) in result.index]

    # Reconstruct potential GDP level with periodic re-anchoring
    start_idx = result["g_Y_potential"].first_valid_index()
    result["log_GDP_potential"] = np.nan

    # Initialize at actual GDP level
    result.loc[start_idx, "log_GDP_potential"] = result.loc[start_idx, "log_GDP_actual"]

    # Track re-anchoring for reporting
    reanchor_log = []

    # Cumulate potential growth with re-anchoring
    for i, idx in enumerate(result.index):
        if idx > start_idx and pd.notna(result.loc[idx, "g_Y_potential"]):
            prev_idx = result.index[result.index.get_loc(idx) - 1]

            # Check if this is a re-anchor point
            if idx in anchor_periods:
                # Re-anchor to actual GDP
                result.loc[idx, "log_GDP_potential"] = result.loc[idx, "log_GDP_actual"]
                gap_before = (
                    result.loc[prev_idx, "log_GDP_actual"] -
                    result.loc[prev_idx, "log_GDP_potential"]
                ) if pd.notna(result.loc[prev_idx, "log_GDP_potential"]) else 0
                reanchor_log.append((str(idx), gap_before))
            elif pd.notna(result.loc[prev_idx, "log_GDP_potential"]):
                # Normal cumulation
                result.loc[idx, "log_GDP_potential"] = (
                    result.loc[prev_idx, "log_GDP_potential"]
                    + result.loc[idx, "g_Y_potential"]
                )

    # Convert back to levels
    result["GDP_actual"] = np.exp(result["log_GDP_actual"] / 100)
    result["GDP_potential"] = np.exp(result["log_GDP_potential"] / 100)

    # Output gap (% deviation from potential)
    result["output_gap"] = (
        (result["GDP_actual"] - result["GDP_potential"])
        / result["GDP_potential"] * 100
    )

    # Summary
    print(f"\nPotential GDP ({method_name} trend MFP):")
    print(f"  Anchor points: {['Start'] + anchor_points}")
    if reanchor_log:
        print("  Re-anchoring adjustments (log gap before reset):")
        for period, gap in reanchor_log:
            print(f"    {period}: {gap:.2f}%")
    print(f"  Mean potential growth: {result['g_Y_potential'].dropna().mean():.3f}% quarterly ({result['g_Y_potential'].dropna().mean()*4:.2f}% annual)")
    print(f"  Mean actual growth:    {result['g_Y_actual'].dropna().mean():.3f}% quarterly ({result['g_Y_actual'].dropna().mean()*4:.2f}% annual)")
    print(f"  Output gap (latest):   {result['output_gap'].dropna().iloc[-1]:.2f}%")
    print(f"  Output gap (std):      {result['output_gap'].dropna().std():.2f}%")

    return result

# Calculate potential GDP using HP-filtered MFP with periodic re-anchoring
potential_hp = calculate_potential_gdp(
    data, growth, mfp_hp_trend,
    method_name="HP",
    anchor_points=ANCHOR_POINTS,
)


Potential GDP (HP trend MFP):
  Anchor points: ['Start', '1990Q1', '2000Q1', '2008Q1', '2019Q4']
  Re-anchoring adjustments (log gap before reset):
    1990Q1: 0.88%
    2000Q1: -0.77%
    2008Q1: -0.22%
    2019Q4: -0.25%
  Mean potential growth: 0.735% quarterly (2.94% annual)
  Mean actual growth:    0.735% quarterly (2.94% annual)
  Output gap (latest):   -0.86%
  Output gap (std):      1.26%


In [14]:
def plot_potential_gdp(
    result: pd.DataFrame,
    method_name: str = "HP",
    anchor_points: list = None,
) -> None:
    """Plot actual vs potential GDP and output gap."""
    if anchor_points is None:
        anchor_points = []
    print(f"Anchor points: {anchor_points}")

    fig, axes = plt.subplots(3, 1, figsize=(9.0, 9.0))

    # Panel 1: GDP levels
    gdp_levels = pd.DataFrame({
        "Actual GDP": result["GDP_actual"],
        "Potential GDP": result["GDP_potential"],
    }).dropna(axis=0, how='any')
    mg.line_plot(
        gdp_levels,
        ax=axes[0],
        color=["blue", "red"],
        width=[1.5, 2],
        style=["solid", "dashed"],
    )
    # Mark anchor points
    label = "Anchor points"
    for ap in anchor_points:
        try:
            ap_period = pd.Period(ap, freq="Q")
            if ap_period in gdp_levels.index:
                axes[0].axvline(x=ap_period.ordinal, color="darkred",
                               linewidth=1, linestyle=":", label=label)
                label = "_"
        except (ValueError, KeyError):
            pass
    axes[0].set_yscale("log")
    mg.finalise_plot(
        axes[0],
        title=f"Actual vs Potential GDP ({method_name} Filter MFP, with re-anchoring)",
        ylabel="GDP (log scale)",
        legend={"loc": "best", "fontsize": "x-small"},
        dont_save=True,
        dont_close=True,
    )

    # Panel 2: GDP growth rates
    growth_rates = pd.DataFrame({
        "Actual Growth": result["g_Y_actual"] * 4,
        "Potential Growth": result["g_Y_potential"] * 4,
    })
    mg.line_plot(
        growth_rates,
        ax=axes[1],
        color=["blue", "red"],
        alpha=[0.5, 1.0],
        width=[1, 2],
        style=["solid", "dashed"],
    )
    mg.finalise_plot(
        axes[1],
        title="GDP Growth: Actual vs Potential (Annualized)",
        ylabel="Growth (% p.a.)",
        y0=True,
        legend={"loc": "best", "fontsize": "x-small"},
        dont_save=True,
        dont_close=True,
    )

    # Panel 3: Output gap
    ax = axes[2]
    gap_df = pd.DataFrame({"Output Gap": result["output_gap"]}).dropna()

    # Use mgplot fill_between_plot
    positive_fill = pd.DataFrame({
        "Zero": 0.0,
        "Positive": gap_df["Output Gap"].clip(lower=0),
    }, index=gap_df.index)
    negative_fill = pd.DataFrame({
        "Negative": gap_df["Output Gap"].clip(upper=0),
        "Zero": 0.0,
    }, index=gap_df.index)
    mg.fill_between_plot(positive_fill, ax=ax, color="green", alpha=0.3, label="Positive gap")
    mg.fill_between_plot(negative_fill, ax=ax, color="red", alpha=0.3, label="Negative gap")

    # Use mgplot line_plot
    mg.line_plot(gap_df, ax=ax, color="black", width=1.5)

    # Mark anchor points
    label = "Anchor points"
    for ap in anchor_points:
        try:
            ap_period = pd.Period(ap)
            if ap_period in gap_df.index:
                ax.axvline(x=ap_period.ordinal, color="darkred",
                          linewidth=1, linestyle=":", label=label)
                label = "_"
        except (ValueError, KeyError):
            pass
    ax.axhline(y=0, color="black", linewidth=1)
    mg.finalise_plot(
        ax,
        title="Output Gap (Actual - Potential) / Potential",
        ylabel="Gap (%)",
        legend={"loc": "best", "fontsize": "x-small"},
        figsize=(9.0, 9.0),
        rfooter="ABS 5206, 1364, 6202",
        lfooter="Australia. Note: Output gap is a mechanical estimate, not informed by inflation dynamics.",
        show=SHOW,
    )

plot_potential_gdp(potential_hp, "HP", anchor_points=ANCHOR_POINTS)

Anchor points: ['1990Q1', '2000Q1', '2008Q1', '2019Q4']


## Growth Decomposition Charts

In [15]:
def plot_growth_decomposition(
    decomp: pd.DataFrame,
    mfp_trend: pd.Series,
    alpha: float = ALPHA,
) -> None:
    """Create stacked bar chart of GDP growth decomposition."""
    # Prepare annual data (sum of quarterly)
    annual = pd.DataFrame({
        "Capital": decomp["contrib_Capital"],
        "Labour (Hours)": decomp["contrib_Hours"],
        "MFP (Trend)": mfp_trend,
    })

    # Group by year and sum
    annual.index = annual.index.to_timestamp()
    annual_sum = annual.groupby(annual.index.year).sum()
    annual_sum = annual_sum.loc[annual_sum.index >= 1986]  # Full years only
    annual_sum = annual_sum.loc[annual_sum.index <= annual_sum.index[-1] - 1]  # Drop incomplete

    # Get actual GDP growth for line
    gdp_annual = decomp["g_GDP"].copy()
    gdp_annual.index = gdp_annual.index.to_timestamp()
    gdp_annual = gdp_annual.groupby(gdp_annual.index.year).sum()
    gdp_annual = gdp_annual.loc[annual_sum.index]

    # Use mgplot for stacked bar
    ax = mg.bar_plot(
        annual_sum,
        stacked=True,
        color=["#1f77b4", "#ff7f0e", "#2ca02c"],
    )

    # Add GDP growth line on top
    ax.plot(annual_sum.index, gdp_annual.values, "ko-", markersize=4, linewidth=1.5, label="Actual GDP Growth")
    ax.axhline(y=0, color="black", linewidth=0.5)

    mg.finalise_plot(
        ax,
        title=f"GDP Growth Decomposition:\nContributions from Capital, Labour, and MFP (α={alpha})",
        ylabel="Growth Contribution (% p.a.)",
        legend={"loc": "best", "fontsize": "x-small"},
        lfooter="Australia. Stacked bars use HP-filtered trend MFP. Bars may not sum exactly to actual GDP (dots) "
                "because the HP filter smooths cyclical MFP fluctuations.",
        rheader="ABS 5206, 1364, 6202",
        show=SHOW,
    )

    # Print summary by decade
    print("\nGDP Growth Decomposition by Decade:")
    print("=" * 60)
    for decade_start in range(1990, 2030, 10):
        decade_end = decade_start + 9
        mask = (annual_sum.index >= decade_start) & (annual_sum.index <= decade_end)
        if mask.sum() > 0:
            decade_data = annual_sum.loc[mask].mean()
            total = gdp_annual.loc[mask].mean()
            print(f"\n{decade_start}s:")
            print(f"  Capital:    {decade_data['Capital']:.2f}%")
            print(f"  Labour:     {decade_data['Labour (Hours)']:.2f}%")
            print(f"  MFP:        {decade_data['MFP (Trend)']:.2f}%")
            print(f"  Total:      {total:.2f}%")

plot_growth_decomposition(decomp, mfp_hp_trend)


GDP Growth Decomposition by Decade:

1990s:
  Capital:    0.88%
  Labour:     0.75%
  MFP:        1.59%
  Total:      3.31%

2000s:
  Capital:    1.34%
  Labour:     1.10%
  MFP:        0.56%
  Total:      3.01%

2010s:
  Capital:    1.11%
  Labour:     1.08%
  MFP:        0.44%
  Total:      2.56%

2020s:
  Capital:    0.55%
  Labour:     1.46%
  MFP:        0.12%
  Total:      2.04%


## Sensitivity Analysis: Capital Share (α)

In [16]:
def sensitivity_analysis_alpha(growth: pd.DataFrame) -> pd.DataFrame:
    """Test sensitivity of MFP to different capital share assumptions."""
    alphas = [0.20, 0.25, 0.30, 0.35, 0.40]
    results = []

    for alpha in alphas:
        # Calculate MFP with this alpha
        mfp = (
            growth["g_GDP"]
            - alpha * growth["g_Capital"]
            - (1 - alpha) * growth["g_Hours"]
        )

        # HP filter
        mfp_trend, _ = apply_hp_filter(mfp.dropna())

        results.append({
            "alpha": alpha,
            "mfp_raw_mean": mfp.dropna().mean(),
            "mfp_trend_mean": mfp_trend.mean(),
            "mfp_recent": mfp_trend.loc["2015Q1":].mean() if len(mfp_trend.loc["2015Q1":]) > 0 else np.nan,
        })

    df = pd.DataFrame(results)

    print("\nSensitivity Analysis: Capital Share (α)")
    print("=" * 60)
    print("\n         Raw MFP      Trend MFP     Recent Trend")
    print("  α     (Q mean)      (Q mean)       (2015+)")
    print("-" * 60)
    for _, row in df.iterrows():
        print(f" {row['alpha']:.2f}    {row['mfp_raw_mean']:.3f}%        {row['mfp_trend_mean']:.3f}%        {row['mfp_recent']:.3f}%")
    print("-" * 60)
    print("Note: Quarterly growth rates (multiply by 4 for annual)")

    return df

sensitivity = sensitivity_analysis_alpha(growth)


Sensitivity Analysis: Capital Share (α)

         Raw MFP      Trend MFP     Recent Trend
  α     (Q mean)      (Q mean)       (2015+)
------------------------------------------------------------
 0.20    0.210%        0.210%        0.082%
 0.25    0.188%        0.188%        0.080%
 0.30    0.166%        0.166%        0.078%
 0.35    0.144%        0.144%        0.076%
 0.40    0.122%        0.122%        0.073%
------------------------------------------------------------
Note: Quarterly growth rates (multiply by 4 for annual)


In [17]:
def plot_sensitivity_alpha(growth: pd.DataFrame) -> None:
    """Plot MFP trends for different alpha values."""
    alphas = [0.25, 0.30, 0.35]

    results = pd.DataFrame()
    for alpha in alphas:
        mfp = (
            growth["g_GDP"]
            - alpha * growth["g_Capital"]
            - (1 - alpha) * growth["g_Hours"]
        )
        mfp_trend, _ = apply_hp_filter(mfp.dropna())
        results[f"α = {alpha}"] = mfp_trend * 4  # Annualize

    mg.line_plot_finalise(
        results,
        title="MFP Trend Sensitivity to Capital Share (α)",
        ylabel="Annual MFP Growth (%)",
        y0=True,
        width=2,
        rfooter="ABS 5206, 1364, 6202",
        lfooter="Australia. ",
        show=SHOW,
    )

plot_sensitivity_alpha(growth)

## Summary Statistics

In [18]:
def print_summary_statistics(
    decomp: pd.DataFrame,
    mfp_hp: pd.Series,
    potential: pd.DataFrame = None,
) -> None:
    """Print comprehensive summary statistics."""
    print("\n" + "=" * 70)
    print("SUMMARY: Cobb-Douglas MFP Decomposition")
    print("=" * 70)

    print("\nModel Parameters:")
    print(f"  Capital share (α):     {ALPHA}")
    print(f"  Labour share (1-α):    {1-ALPHA}")
    print(f"  HP filter lambda:      {HP_LAMBDA}")

    print("\nSample Period:")
    idx = decomp["g_MFP_raw"].dropna().index
    print(f"  {idx[0]} to {idx[-1]} ({len(idx)} quarters)")

    print("\nMean Annual Growth Rates:")
    print(f"  {'Component':<20} {'Full Sample':>12} {'1990-1999':>12} {'2000-2009':>12} {'2010-2019':>12} {'2020+':>12}")
    print(f"  {'-'*20} {'-'*12} {'-'*12} {'-'*12} {'-'*12} {'-'*12}")

    components = {
        "GDP": decomp["g_GDP"],
        "Capital contrib.": decomp["contrib_Capital"],
        "Labour contrib.": decomp["contrib_Hours"],
        "MFP (raw)": decomp["g_MFP_raw"],
        "MFP (HP trend)": mfp_hp,
    }

    periods = [
        (None, None),  # Full sample
        ("1990Q1", "1999Q4"),
        ("2000Q1", "2009Q4"),
        ("2010Q1", "2019Q4"),
        ("2020Q1", None),
    ]

    for name, series in components.items():
        values = []
        for start, end in periods:
            s = series.dropna()
            if start:
                s = s[s.index >= pd.Period(start)]
            if end:
                s = s[s.index <= pd.Period(end)]
            if len(s) > 0:
                values.append(f"{s.mean() * 4:>11.2f}%")
            else:
                values.append(f"{'n/a':>12}")
        print(f"  {name:<20} {' '.join(values)}")

    if potential is not None:
        print(f"\nOutput Gap (latest): {potential['output_gap'].dropna().iloc[-1]:.2f}%")

    print("\n" + "=" * 70)

print_summary_statistics(decomp, mfp_hp_trend, potential_hp)


SUMMARY: Cobb-Douglas MFP Decomposition

Model Parameters:
  Capital share (α):     0.3
  Labour share (1-α):    0.7
  HP filter lambda:      1600

Sample Period:
  1985Q2 to 2025Q3 (162 quarters)

Mean Annual Growth Rates:
  Component             Full Sample    1990-1999    2000-2009    2010-2019        2020+
  -------------------- ------------ ------------ ------------ ------------ ------------
  GDP                         2.94%        3.31%        3.01%        2.56%        2.04%
  Capital contrib.            1.05%        0.88%        1.34%        1.11%        0.56%
  Labour contrib.             1.22%        0.75%        1.10%        1.08%        1.37%
  MFP (raw)                   0.66%        1.68%        0.58%        0.37%        0.11%
  MFP (HP trend)              0.66%        1.59%        0.56%        0.44%        0.06%

Output Gap (latest): -0.86%



## Phillips Curve Cross-Check

Compare the output gap from the production function approach with inflation dynamics.
A sensible output gap should show some relationship with inflation:
- Positive output gaps → above-target inflation
- Negative output gaps → below-target inflation

This cross-check uses trimmed mean inflation (core CPI) as the inflation measure.

### Note: Inflation-Disciplined MFP Approach Abandoned

We explored using the Phillips Curve relationship to "discipline" the output gap estimate and back-calculate an inflation-consistent MFP. The idea was to invert the Phillips Curve: if inflation deviates from target, infer what the output gap must have been, then derive the implied MFP.

**This approach was abandoned** because the Phillips Curve relationship is highly unstable over time:
- **2010s**: Essentially flat - inflation stayed 1.5-3% regardless of unemployment (5-6%)
- **1998-2002**: No coherent relationship
- **2003-2008**: Some slope, but shifts around
- **2021-2025**: Only recent period shows a clear negative slope

Without a stable Phillips Curve slope, inverting the relationship produces unreliable results. The joint NAIRU+Output Gap model handles this better by *estimating* the relationship simultaneously rather than assuming a fixed slope.

In [19]:
def get_trimmed_mean_inflation() -> tuple[pd.Series, pd.Series]:
    """Get trimmed mean inflation from ABS CPI data.
    
    Returns quarterly and annual trimmed mean inflation.
    """
    # Trimmed mean identifiers from ABS CPI
    tm_quarterly = "Percentage Change from Previous Period ;  Trimmed Mean ;  Australia ;"
    tm_annual = "Percentage Change from Corresponding Quarter of Previous Year ;  Trimmed Mean ;  Australia ;"
    old_cpi = "./ABS-Data/Qrtly-CPI-Time-series-spreadsheets-all.zip"

    wanted = {
        "Trimmed Mean Quarterly":
            ReqsTuple("", "640106", tm_quarterly, "S", "", False, False, old_cpi),
        "Trimmed Mean Annual":
            ReqsTuple("", "640106", tm_annual, "S", "", False, False, old_cpi),
    }

    inflation_data = get_abs_data_structured(wanted)

    quarterly_inflation = inflation_data["Trimmed Mean Quarterly"]
    quarterly_inflation.name = "Trimmed Mean (Q/Q)"

    annual_inflation = inflation_data["Trimmed Mean Annual"]
    annual_inflation.name = "Trimmed Mean (Y/Y)"

    print(f"Trimmed mean inflation data: {quarterly_inflation.index[0]} to {quarterly_inflation.index[-1]}")
    print(f"Latest annual inflation: {annual_inflation.iloc[-1]:.1f}%")

    return quarterly_inflation, annual_inflation


trimmed_mean_qtr, trimmed_mean_ann = get_trimmed_mean_inflation()

Trimmed mean inflation data: 1948Q3 to 2025Q3
Latest annual inflation: 3.0%


In [20]:
def plot_phillips_curve_crosscheck(
    output_gap: pd.Series,
    inflation_annual: pd.Series,
    inflation_qtr: pd.Series,
) -> pd.DataFrame:
    """Plot Phillips Curve cross-check: output gap vs inflation.

    Creates multiple views:
    1. Time series comparison (output gap and inflation)
    2. Scatter plot with fitted line
    3. Inflation deviation from target vs output gap
    """
    # Align data to common sample
    common = pd.DataFrame({
        "Output Gap": output_gap,
        "Inflation (Y/Y)": inflation_annual,
        "Inflation (Q/Q)": inflation_qtr,
    }).dropna()

    # Inflation deviation from 2.5% target
    common["Inflation Deviation"] = common["Inflation (Y/Y)"] - 2.5

    print(f"Cross-check sample: {common.index[0]} to {common.index[-1]} ({len(common)} obs)")

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1. Time series: Output gap and inflation (twin axes)
    ax = axes[0, 0]
    ax2 = ax.twinx()

    gap_df = pd.DataFrame({"Output Gap": common["Output Gap"]})
    inflation_df = pd.DataFrame({"Trimmed Mean Inflation": common["Inflation (Y/Y)"]})

    # Use mgplot for output gap (fill_between and line)
    positive_fill = pd.DataFrame({
        "Zero": 0.0,
        "Positive": gap_df["Output Gap"].clip(lower=0),
    }, index=gap_df.index)
    negative_fill = pd.DataFrame({
        "Negative": gap_df["Output Gap"].clip(upper=0),
        "Zero": 0.0,
    }, index=gap_df.index)
    max_ticks=8
    mg.fill_between_plot(positive_fill, ax=ax, color="green", alpha=0.3, max_ticks=max_ticks)
    mg.fill_between_plot(negative_fill, ax=ax, color="red", alpha=0.3, max_ticks=max_ticks)
    mg.line_plot(gap_df, ax=ax, color="black", width=1.5, max_ticks=max_ticks)

    ax.axhline(y=0, color="black", linewidth=0.5)
    ax.set_ylabel("Output Gap (%)", color="black")
    ax.set_ylim(-8, 4)

    # Use mgplot for inflation on twin axis
    mg.line_plot(inflation_df, ax=ax2, color="darkorange", width=2, max_ticks=max_ticks)
    ax2.axhline(y=2.5, color="darkorange", linewidth=1, linestyle="--", alpha=0.7)
    ax2.axhspan(2.0, 3.0, color="orange", alpha=0.1)
    ax2.set_ylabel("Annual Inflation (%)", color="darkorange")
    ax2.set_ylim(0, 8)

    ax.set_title("Output Gap vs Core Inflation\n(Time Series)")
    ax.legend(loc="upper left", fontsize="x-small")
    ax2.legend(loc="upper right", fontsize="x-small")

    # 2. Scatter plot: Output gap vs inflation level (keep matplotlib)
    ax = axes[0, 1]
    ax.scatter(common["Output Gap"], common["Inflation (Y/Y)"],
               alpha=0.5, s=30, c="blue")

    # Fit line
    z = np.polyfit(common["Output Gap"], common["Inflation (Y/Y)"], 1)
    p = np.poly1d(z)
    x_line = np.linspace(common["Output Gap"].min(), common["Output Gap"].max(), 100)
    ax.plot(x_line, p(x_line), "r-", linewidth=2, label=f"Fitted: slope={z[0]:.2f}")

    ax.axhline(y=2.5, color="gray", linewidth=1, linestyle="--", alpha=0.7)
    ax.axvline(x=0, color="gray", linewidth=1, linestyle="--", alpha=0.7)
    ax.set_xlabel("Output Gap (%)")
    ax.set_ylabel("Annual Trimmed Mean Inflation (%)")
    ax.set_title("Phillips Curve:\nOutput Gap vs Inflation Level")
    ax.legend(fontsize="x-small")

    # 3. Scatter: Output gap vs inflation deviation from target (keep matplotlib)
    ax = axes[1, 0]
    ax.scatter(common["Output Gap"], common["Inflation Deviation"],
               alpha=0.5, s=30, c="darkgreen")

    # Fit line for deviation
    z2 = np.polyfit(common["Output Gap"], common["Inflation Deviation"], 1)
    p2 = np.poly1d(z2)
    ax.plot(x_line, p2(x_line), "r-", linewidth=2, label=f"Fitted: slope={z2[0]:.2f}")

    ax.axhline(y=0, color="gray", linewidth=1, linestyle="--", alpha=0.7)
    ax.axvline(x=0, color="gray", linewidth=1, linestyle="--", alpha=0.7)
    ax.set_xlabel("Output Gap (%)")
    ax.set_ylabel("Inflation Deviation from 2.5% Target")
    ax.set_title("Phillips Curve:\nOutput Gap vs Inflation Deviation")
    ax.legend(fontsize="x-small")

    # 4. Rolling correlation - use mgplot
    rolling_corr = common["Output Gap"].rolling(window=20).corr(common["Inflation Deviation"])
    corr_data = pd.DataFrame({"Rolling Correlation": rolling_corr})

    mg.line_plot(corr_data, ax=axes[1, 1], color="purple", width=1.5, max_ticks=8)
    axes[1, 1].axhline(y=0, color="black", linewidth=0.5)
    axes[1, 1].axhline(y=rolling_corr.mean(), color="gray", linewidth=1, linestyle="--")
    axes[1, 1].set_ylim(-1, 1)

    mg.finalise_plot(
        axes[1, 1],
        title="Rolling 5-Year Correlation:\nOutput Gap vs Inflation Deviation",
        ylabel="Correlation",
        figsize=(9.0, 9.0),
        rfooter="ABS 5206, 1364, 6401",
        lfooter="Australia. ",
        show=SHOW,
    )

    # Print correlation statistics
    corr = common["Output Gap"].corr(common["Inflation Deviation"])
    corr_level = common["Output Gap"].corr(common["Inflation (Y/Y)"])

    print("\nPhillips Curve Cross-Check Statistics:")
    print(f"  Correlation (gap vs inflation level):     {corr_level:.3f}")
    print(f"  Correlation (gap vs inflation deviation): {corr:.3f}")
    print(f"  Slope (gap → inflation deviation):        {z2[0]:.3f}")

    # Interpretation
    if corr > 0.3:
        print("\n  ✓ Positive relationship suggests output gap has some Phillips Curve validity")
    elif corr < -0.1:
        print("\n  ✗ Negative relationship - output gap may be poorly identified")
    else:
        print("\n  ~ Weak relationship - output gap estimates may not capture demand pressure well")

    return common


phillips_data = plot_phillips_curve_crosscheck(
    potential_hp["output_gap"],
    trimmed_mean_ann,
    trimmed_mean_qtr,
)

Cross-check sample: 1985Q2 to 2025Q3 (162 obs)

Phillips Curve Cross-Check Statistics:
  Correlation (gap vs inflation level):     0.163
  Correlation (gap vs inflation deviation): 0.163
  Slope (gap → inflation deviation):        0.253

  ~ Weak relationship - output gap estimates may not capture demand pressure well


### Interpretation

This chart tests whether the output gap from the Cobb-Douglas model makes economic sense by checking it against inflation (the Phillips Curve relationship).

**What each panel shows:**

1. **Top-left (Time Series)**: Output gap (black/shaded) vs trimmed mean inflation (orange). The output gap has been mostly negative since ~1990, yet inflation has varied widely - no obvious correlation.

2. **Top-right & Bottom-left (Scatter plots)**: Each dot is a quarter. The fitted line has a slope of only ~0.25 - meaning a 1% larger output gap is associated with only 0.25% higher inflation. Very weak.

3. **Bottom-right (Rolling correlation)**: The 5-year rolling correlation between output gap and inflation deviation swings wildly from -0.8 to +0.8 over time - no stable relationship.

**The message:**

The output gap from this model has a **weak, unstable relationship** with inflation. A "good" output gap estimate should show:
- Positive gaps → above-target inflation
- Negative gaps → below-target inflation

But here:
- Correlation is only ~0.16 (very weak)
- The relationship flips sign across different periods

**Implication**: This output gap is "notional only". It's derived purely from filtered productivity, not disciplined by inflation outcomes. For policy-relevant output gap estimates, prefer models that incorporate inflation information (like joint NAIRU+Output Gap models).

## Export Results

In [21]:
def export_results(
    decomp: pd.DataFrame,
    mfp_hp: pd.Series,
    potential: pd.DataFrame | None = None,
    inflation_qtr: pd.Series | None = None,
    inflation_ann: pd.Series | None = None,
) -> None:
    """Export key results to CSV."""
    output = pd.DataFrame({
        "g_GDP": decomp["g_GDP"],
        "g_Capital": decomp["g_Capital"],
        "g_Hours": decomp["g_Hours"],
        "contrib_Capital": decomp["contrib_Capital"],
        "contrib_Hours": decomp["contrib_Hours"],
        "MFP_raw": decomp["g_MFP_raw"],
        "MFP_HP_trend": mfp_hp,
    })

    if potential is not None:
        output["g_Y_potential"] = potential["g_Y_potential"]
        output["output_gap"] = potential["output_gap"]

    if inflation_qtr is not None:
        output["inflation_trimmed_mean_qtr"] = inflation_qtr

    if inflation_ann is not None:
        output["inflation_trimmed_mean_ann"] = inflation_ann

    output_path = "./MODEL_OUTPUTS/cobb_douglas_mfp_decomposition.csv"
    output.to_csv(output_path)
    print(f"Results exported to: {output_path}")

export_results(decomp, mfp_hp_trend, potential_hp, trimmed_mean_qtr, trimmed_mean_ann)

Results exported to: ./MODEL_OUTPUTS/cobb_douglas_mfp_decomposition.csv


## Finished

In [22]:
# Watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-12-13 15:15:53

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.8.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

statsmodels: 0.14.6
numpy      : 2.3.5
readabs    : 0.1.8
mgplot     : 0.2.14
pandas     : 2.3.3
matplotlib : 3.10.7

Watermark: 2.5.0



In [23]:
print("Finished")

Finished
